# Import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.signal import savgol_filter

# Read data

In [ ]:
# Read the CSV file
df = pd.read_csv(
    "/home/m3rc7pi/workspace/arc/log/20240324_test.csv", 
    header=None, 
    names=["timestamp", "acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"],
    skiprows=0,
)
df['timestamp'] = df['timestamp'] / 1e6

In [ ]:

df["delta_time"] = df['timestamp'].diff() * 1000
Fs = 1 / (df["delta_time"][2:].mean()/1000)  # Sampling frequency (Hz)

plt.figure(figsize=(12, 3))
plt.plot(df['timestamp'][2:], df["delta_time"][2:])  # Plot only positive frequencies
plt.xlabel('Time (s)')
plt.ylabel('dt [ms]')
plt.title('Spectral Analysis of acc_x')
plt.grid(True)
plt.show()

print(df["delta_time"][2:].describe())
print("\nAverage sampling frequency:", Fs, "[Hz]")

# Accelerometer

In [ ]:
def magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

def plot_acc(x:str, y:str, z:str, figsize=(12, 6) ):
    fig, axs = plt.subplots(2, 1, figsize=figsize)
    axs[0].plot(df['timestamp'], df[x], label=x)
    axs[0].plot(df['timestamp'], df[y], label=y)
    axs[0].plot(df['timestamp'], df[z], label=z)
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel('Acceleration [m/s]')
    axs[0].set_title('Accelerometer Data')
    axs[0].grid(True)
    axs[0].legend()

    axs[1].plot( magnitude(df[x],df[y], df[z]))
    axs[1].set_xlabel('Time [s]')
    axs[1].set_title('Magnitude')
    axs[1].grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
df['acc_magnitude'] = magnitude(df['acc_x'], df['acc_y'], df['acc_z'])

plot_acc('acc_x', 'acc_y', 'acc_z')
print(df[["acc_x", "acc_y", "acc_z", 'acc_magnitude']].describe())

In [ ]:
def process_data(axis, window_length=51, polyorder=3):
    df[axis + '_filtered'] = savgol_filter(df[axis], window_length=window_length, polyorder=polyorder)

    # Compute the difference between acc_x and acc_x_filtered
    df[axis + '_noise'] = df[axis] - df[axis + '_filtered']

    # Perform FFT on acc_x
    df[axis + '_fft'] = np.fft.fft(df[axis])
    df[axis + '_freq']  = np.fft.fftfreq(len(df[axis]), d=1/Fs)  # Frequency bins

    # Perform FFT on acc_x_filtered
    df[axis + '_filtered_fft'] = np.fft.fft(df[axis + '_filtered'])
    df[axis + '_filtered_freq'] = np.fft.fftfreq(len(df[axis + '_filtered']), d=1/Fs)  # Frequency bins

def plot_data_filtered(axis, y_label, figsize=(12, 8)):
    fig, axs = plt.subplots(4, 1, figsize=figsize)

    axs[0].plot(df['timestamp'], df[axis], label=axis)
    axs[0].plot(df['timestamp'], df[axis + '_filtered'], label=axis + '_filtered')
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel(y_label)
    axs[0].set_title('Filtered ' + axis)
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(df['timestamp'], df[axis + '_noise'], label=axis + '_noise')
    axs[1].set_xlabel('Time [s]')
    axs[1].set_ylabel(y_label)
    axs[1].set_title('Noise of '+ axis)
    axs[1].set_ylim(-0.5, 0.5)
    axs[1].legend()
    axs[1].grid()

    dim = len(df[axis + '_freq'])//2
    axs[2].plot(df[axis + '_freq'][0:dim], np.abs(df[axis + '_fft'][0:dim]))  # Plot only positive frequencies
    axs[2].set_xlabel('Frequency [Hz]')
    axs[2].set_ylabel('Amplitude')
    axs[2].set_title('Spectrum of ' + axis)
    axs[2].grid()

    filt_dim = len(df[axis + '_filtered_freq'])//2
    axs[3].plot(df[axis + '_filtered_freq'][0:filt_dim], np.abs(df[axis + '_filtered_fft'][0:filt_dim]))
    axs[3].set_xlabel('Frequency [Hz]')
    axs[3].set_ylabel('Amplitude')
    axs[3].set_title('Spectrum of ' + axis + 'Filtered')
    axs[3].grid()

    plt.tight_layout()
    plt.show()

In [ ]:
process_data('acc_x', window_length=51, polyorder=3)
plot_data_filtered('acc_x', y_label='[m/s]')

print(df[["acc_x", "acc_x_filtered", "acc_x_noise"]].describe())

In [ ]:
process_data('acc_y', window_length=51, polyorder=3)
plot_data_filtered('acc_y', y_label='[m/s]')

print(df[["acc_y", "acc_y_filtered", "acc_y_noise"]].describe())

In [ ]:
process_data('acc_z', window_length=51, polyorder=3)
plot_data_filtered('acc_z', y_label='[m/s]')

print(df[["acc_z", "acc_z_filtered", "acc_z_noise"]].describe())

# Gyroscope

In [ ]:
def plot_gyro(x:str, y:str, z:str, figsize=(12, 3)):
    plt.figure(figsize=figsize)
    plt.plot(df['timestamp'], df[x], label=x)
    plt.plot(df['timestamp'], df[y], label=y)
    plt.plot(df['timestamp'], df[z], label=z)
    plt.xlabel('Timestamp [s]')
    plt.ylabel('Angular Velocity [rad/s]')
    plt.title('Gyroscope Data')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_gyro("gyro_x", "gyro_y", "gyro_z")
print(df[["acc_x", "acc_y", "acc_z", 'acc_magnitude']].describe())

In [ ]:
process_data('gyro_x', window_length=51, polyorder=3)
plot_data_filtered('gyro_x', y_label='[rad/s]')

print(df[["gyro_x", "gyro_x_filtered", "gyro_x_noise"]].describe())

In [ ]:
process_data('gyro_y', window_length=51, polyorder=3)
plot_data_filtered('gyro_y', y_label='[rad/s]')

print(df[["gyro_y", "gyro_y_filtered", "gyro_y_noise"]].describe())

In [ ]:
process_data('gyro_z', window_length=51, polyorder=3)
plot_data_filtered('gyro_z', y_label='[rad/s]')

print(df[["gyro_z", "gyro_z_filtered", "gyro_z_noise"]].describe())